**Table of contents**<a id='toc0_'></a>    
- [Import Statements](#toc1_1_)    
- [Record linkage](#toc2_)    
  - [Preprocessing](#toc2_1_)    
  - [Indexing](#toc2_2_)    
  - [Comparing](#toc2_3_)    
  - [Classification](#toc2_4_)    
- [Record linkage workflow summary](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=5
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_'></a>[Import Statements](#toc0_)

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadr
import datetime as pydt
import missingno as msno
from thefuzz import fuzz, process
import recordlinkage

## <a id='toc2_'></a>[Record linkage](#toc0_)

The term record linkage is used to indicate the procedure of bringing together information from two or more records that are believed to belong to the same entity. Record linkage is used to link data from multiple data sources or to find duplicates in a single data source. In computer science, record linkage is also known as data matching, data linking, entity resolution, or field matching.

Dataset A

Name     |   Age    |   Address
---------|----------|---------------
Alice    |   28     |   123 Main St
Bob      |   32     |   456 Elm St
Charlie  |   22     |   789 Oak St

Dataset B


person_name     |    age_years    |   street_address
---------|----------|--------------------------------
Allan    |   28     |   123 Main Street
Brian    |   30     |   457 Elm Street
Dave     |   22     |   789 Oak St


If you pay close attention to the above two datasets, you will notice that the column names are slightly different. Also, although the "Names" are different for each entry in both datasets, the "Age" and "Address" entries for the first and third entries are practically the same (with slight difference in "Address" strings but we can clearly understand that they are the same). This can be termed as "potential matches or duplicates based on certain similarity criteria". 

If we wanted to merge these two datasets we wouldn't want to create duplicate entries in the new dataset. In such complicated merging scenarios, `pd.merge()` is not enough. We need to use record linkage techniques to identify the potential matches and then merge the datasets.

The record linkage procedure can be represented as a **workflow**. The steps are:
 
0. cleaning 
1. indexing 
2. comparing 
3. classifying 
4. evaluation

If needed, the classified record pairs flow back to improve the previous step.

The `recordlinkage` package in Python provides a simple interface to link records in or between data sources and follows the above workflow. 

### <a id='toc2_1_'></a>[Preprocessing](#toc0_)

Preprocessing data, like cleaning and standardising, may increase record linkage accuracy. Pandas is a very good package for data cleaning and preprocessing. The `recordlinkage` package also provides some preprocessing functions. Such as: `recordlinkage.preprocessing.clean()`, `recordlinkage.preprocessing.phonenumbers()` etc. See the [documentation](https://recordlinkage.readthedocs.io/en/latest/ref-preprocessing.html) for more information.

### <a id='toc2_2_'></a>[Indexing](#toc0_)

The indexing module is used to make pairs of records. These pairs are called candidate links or candidate matches. There are `several indexing algorithms available such as full, blocking and sorted neighborhood indexing`. See the [documentation](https://recordlinkage.readthedocs.io/en/latest/ref-index.html) for detailed information on implementation of each indexing algorithm.

Before jumping into coding let's first familiarize ourselves with some of the relevant terminologies.

*`Record pair:`* A record pair refers to a pair of records or data points from two different datasets that are considered potential matches or duplicates based on certain similarity criteria. 

Record linkage or entity resolution tasks often involve identifying record pairs that represent the same entity in different datasets, despite variations or discrepancies in the data.

*`Full record space:`* A full record space is the set of all possible record pairs between two datasets.

**For example**, if we consider dataset A and dataset B, to find all possible record pairs we would match each record in dataset A with each record in dataset B in a combinatory fashion. This would result in 9 record pairs.

The number of record pairs in a full record space is given by, $$No.\ of\ records\ in\ dataset\ A\ X\ No.\ of\ records\ in\ dataset\ B$$

This doesn't depend on the number of columns rather it depends on the number of records in each dataset. This can be helpful to identify duplicates with more accuracy but for large datasets this is computationally expensive (two datasets of 1000 rows will generate a 1 million record pairs).

*`Blocking:`* Blocking is a common technique used in record linkage to reduce the number of record pairs that need to be compared. It involves grouping records into blocks or buckets based on some common attribute(s) and certain criteria, to limit the number of potential comparisons. This is somewhat similar to groupby in pandas.

**For example**, when blocking is applied to the "Address" column (with partial matching constraint), the process would look like this:

Block 1

Name     |   Age    |   Address
---------|----------|----------------------
Alice    |   28     |   123 Main St
Allan    |   28     |   123 Main Street

Block 2

Name     |   Age    |   Address
---------|----------|----------------------
Bob      |   32     |   456 Elm St
Brian    |   30     |   457 Elm Street

Block 3

Name     |   Age    |   Address
---------|----------|----------------------
Charlie  |   22     |   789 Oak St
Dave     |   22     |   789 Oak St

Now, instead of comparing every record in Dataset A with every record in Dataset B, you would only compare records within the same blocks. This reduces the number of potential record pairs to consider for comparison, making the record linkage process more efficient.

**Note:** It's important to choose blocking criteria that are likely to group together records that are more likely to be matches, which can further improve the efficiency of the record linkage process.

In [3]:
from recordlinkage.datasets import load_febrl4

In [4]:
df_A, df_B = load_febrl4()

In [7]:
df_A.head(3)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168


In [8]:
df_A.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-1070-org to rec-66-org
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     4888 non-null   object
 1   surname        4952 non-null   object
 2   street_number  4842 non-null   object
 3   address_1      4902 non-null   object
 4   address_2      4580 non-null   object
 5   suburb         4945 non-null   object
 6   postcode       5000 non-null   object
 7   state          4950 non-null   object
 8   date_of_birth  4906 non-null   object
 9   soc_sec_id     5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


In [6]:
df_B.head(3)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855


In [9]:
df_B.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-561-dup-0 to rec-493-dup-0
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     4766 non-null   object
 1   surname        4898 non-null   object
 2   street_number  4713 non-null   object
 3   address_1      4780 non-null   object
 4   address_2      4149 non-null   object
 5   suburb         4894 non-null   object
 6   postcode       5000 non-null   object
 7   state          4893 non-null   object
 8   date_of_birth  4801 non-null   object
 9   soc_sec_id     5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


> The Python Record Linkage Toolkit contains basic and advanced indexing (or blocking) algorithms to make record pairs. The algorithms are Python classes. Popular algorithms in the toolkit are: `recordlinkage.index.Full`, `recordlinkage.index.Block`, `recordlinkage.index.SortedNeighbourhood`.

In [11]:
# first we create an indexer object
indexer = recordlinkage.Index()

# now we will set the indexing technique as blocking for creating record pairs
indexer.block(left_on="given_name", right_on="given_name")

# generating the record pairs for df_A and df_B
possible_pairs = indexer.index(df_A, df_B)

In [14]:
len(df_A), len(df_B), len(possible_pairs)

(5000, 5000, 77249)

It is possible to parse a list of columns names to block on multiple variables. Blocking on multiple variables will reduce the number of record pairs even further. The resulting object, is a pandas multi-index object containing pairs of row indices from both DataFrames, which is a fancy way to say it is an array containing possible pairs of indices that makes it much easier to subset DataFrames on. 

In [15]:
possible_pairs

MultiIndex([('rec-1070-org', 'rec-3024-dup-0'),
            ('rec-1070-org', 'rec-2371-dup-0'),
            ('rec-1070-org', 'rec-4652-dup-0'),
            ('rec-1070-org', 'rec-4795-dup-0'),
            ('rec-1070-org', 'rec-1314-dup-0'),
            ('rec-2371-org', 'rec-3024-dup-0'),
            ('rec-2371-org', 'rec-2371-dup-0'),
            ('rec-2371-org', 'rec-4652-dup-0'),
            ('rec-2371-org', 'rec-4795-dup-0'),
            ('rec-2371-org', 'rec-1314-dup-0'),
            ...
            ('rec-1413-org', 'rec-1413-dup-0'),
            ( 'rec-735-org',  'rec-735-dup-0'),
            ('rec-1361-org', 'rec-1361-dup-0'),
            ('rec-3090-org', 'rec-3090-dup-0'),
            ('rec-2571-org', 'rec-2571-dup-0'),
            ('rec-4528-org', 'rec-4528-dup-0'),
            ('rec-4887-org', 'rec-4887-dup-0'),
            ('rec-4350-org', 'rec-4350-dup-0'),
            ('rec-4569-org', 'rec-4569-dup-0'),
            ('rec-3125-org', 'rec-3125-dup-0')],
           names=['rec_

### <a id='toc2_3_'></a>[Comparing](#toc0_)

Each record pair is a candidate match. A set of informative, discriminating and independent features is important for a good classification of record pairs into matching and distinct pairs. To classify the candidate record pairs into matches and non-matches, compare the record pairs on all attributes both datasets have in common. 

The `recordlinkage.Compare` class and its methods can be used to compare records pairs. The `Compare` class has methods like `string, exact, geo, date` and `numeric` and are used for defining the algorithm to use along with the attribute(s) to which the algorithm is applied. The `compute` method is used to start the actual comparing. See the [documentation](https://recordlinkage.readthedocs.io/en/latest/ref-compare.html) for details.

<u>Comparison Algorithms</u>
- *recordlinkage.compare.Exact(left_on, right_on, agree_value=1, disagree_value=0, missing_value=0, label=None):* Compare the record pairs on exact equality of the attributes.

- *recordlinkage.compare.String(left_on, right_on, method='levenshtein', threshold=None, missing_value=0.0, label=None):* Compute the (partial) similarity between string values.

- *recordlinkage.compare.Numeric(left_on, right_on, method='linear', offset=0.0, scale=1.0, origin=0.0, missing_value=0.0, label=None):* Compute the (partial) similarity between numeric values.

- *recordlinkage.compare.Geographic(left_on_lat, left_on_lng, right_on_lat, right_on_lng, method=None, offset=0.0, scale=1.0, origin=0.0, missing_value=0.0, label=None):* Compute the (partial) similarity between WGS84 coordinates.

- *recordlinkage.compare.Date(left_on, right_on, swap_month_day=0.5, swap_months='default', errors='coerce', missing_value=0.0, label=None):* Compute the (partial) similarity between dates.

The `label` argument in each of these class objects lets us set the name of the column in the resulting DataFrame.

In [ ]:
"date_of_birth", "suburb", "state"

In [27]:
# first we create a comparator object
comparator = recordlinkage.Compare()

# then we define the comparison algorithm for each of the attributes we want to compare the record pairs on
comparator.exact(left_on="given_name", right_on="given_name", label="given_name")
comparator.exact(left_on="date_of_birth", right_on="date_of_birth", label="date_of_birth")
comparator.exact(left_on="suburb", right_on="suburb", label="suburb")
comparator.exact(left_on="state", right_on="suburb", label="suburb")

comparator.string(left_on="surname", right_on="surname", threshold=0.85, label="surname")
comparator.string(left_on="address_1", right_on="address_1", threshold=0.85, label="address_1")

<Compare>

In [28]:
# the actual scoring is initiated by calling the compute method
# the compute method is called with (record_pairs, left_dataset, right_dataset)
features = comparator.compute(possible_pairs, df_A, df_B)

A DataFrame is returned in which the record pairs are stored vertically while all features (comparison scores of the attributes) for each record pair are stored horizontally.

In [29]:
features

given_name  date_of_birth  suburb  suburb  \
rec_id_1     rec_id_2                                                    
rec-1070-org rec-3024-dup-0           1              0       0       0   
             rec-2371-dup-0           1              0       0       0   
             rec-4652-dup-0           1              0       0       0   
             rec-4795-dup-0           1              0       0       0   
             rec-1314-dup-0           1              0       0       0   
...                                 ...            ...     ...     ...   
rec-4528-org rec-4528-dup-0           1              1       1       0   
rec-4887-org rec-4887-dup-0           1              1       0       0   
rec-4350-org rec-4350-dup-0           1              1       1       0   
rec-4569-org rec-4569-dup-0           1              1       1       0   
rec-3125-org rec-3125-dup-0           1              1       0       0   

                             surname  address_1  
rec_id_1     rec_id_2                            
rec-1070-org rec-3024-dup-0      0.0        0.0  
             rec-2371-dup-0      0.0        0.0  
             rec-4652-dup-0      0.0        0.0  
             rec-4795-dup-0      0.0        0.0  
             rec-1314-dup-0      0.0        0.0  
...                              ...        ...  
rec-4528-org rec-4528-dup-0      1.0        1.0  
rec-4887-org rec-4887-dup-0      1.0        1.0  
rec-4350-org rec-4350-dup-0      1.0        1.0  
rec-4569-org rec-4569-dup-0      1.0        0.0  
rec-3125-org rec-3125-dup-0      1.0        1.0  

[77249 rows x 6 columns]

### <a id='toc2_4_'></a>[Classification](#toc0_)

## <a id='toc3_'></a>[Record linkage workflow summary](#toc0_)

The record linkage workflow involves a series of well-defined steps, from data preparation and blocking to comparison, classification, evaluation, and potential postprocessing. The aim is to identify record pairs that likely represent the same entities across two datasets while minimizing false positives and false negatives. The choice of methods and thresholds may vary depending on the specific requirements of the record linkage task.

1. **Preprocessing:**
   - Data Preparation: Clean and preprocess the datasets to ensure consistency and uniformity. This may involve standardizing formats, handling missing values, and removing duplicates within each dataset.
   - Data Transformation: Convert and format the data, if necessary, to ensure compatibility between the two datasets. Ensure that the data types and structures align.

2. **Indexing (Blocking):**
   - Blocking: Create blocks or groups of records within each dataset based on specific attributes or criteria. The goal is to reduce the number of record pairs that need to be compared. Common blocking methods include sorting, hashing, or using range-based criteria, such as the first few characters of an attribute like "Address."

3. **Comparing:**
   - Comparison: Within each block, compare the records based on selected similarity criteria. This involves comparing attribute values (e.g., "Name," "Age") and applying similarity metrics such as string distance measures (e.g., Levenshtein distance) for text fields.
   - Scoring: Assign a similarity score to each pair of records within the block to quantify how similar they are based on the chosen criteria.

4. **Classifying (Thresholding):**
   - Thresholding: Define a threshold for the similarity score above which record pairs are considered potential matches. Record pairs with similarity scores below this threshold are classified as non-matches.
   - Classification: Based on the threshold, classify the record pairs within each block into two categories: matches and non-matches.

5. **Evaluation:**
   - Ground Truth: If available, use a ground truth or labeled dataset that indicates which record pairs are true matches and non-matches. This is used for evaluating the accuracy of your record linkage process.
   - Performance Metrics: Calculate performance metrics such as precision, recall, F1-score, and accuracy to assess the quality of the record linkage results.
   - Iteration: Depending on the evaluation results, you may iterate on the threshold or the choice of similarity criteria to optimize the linkage process for your specific use case.

6. **Postprocessing (Optional):**
   - If needed, perform postprocessing steps to further refine the matched results. This might involve handling cases of potential duplicates within the matched pairs or dealing with data integration challenges.
